In [ ]:
import random
import time
import numpy as np


def quick_pivot_first(arr):
    if len(arr) == 0:
        return []
    pivot = arr[0]
    left = []
    right = []
    
    left = list(filter(lambda x: x < pivot, arr))
    right = list(filter(lambda x: x > pivot, arr))
    
    return quick_pivot_first(left) + [pivot] + quick_pivot_first(right)

def partition(arr, left, right):
    pivot = arr[right]
    index = left
    for i in range(left, right):
        if arr[i] <= pivot:
            arr[i], arr[index] = arr[index], arr[i]
            index += 1
    arr[index], arr[right] = arr[right], arr[index]
    return index

def partition_random(arr, left, right):
    n = random.randint(left, right)
    arr[n], arr[right] = arr[right], arr[n]
    return partition(arr, left, right)

def quick_random(arr, low, high):
    if low >= high:
        return []
    left = []
    right = []

    index = random.randint(low, high)
    #print(low, high, index)
    pivot = arr[index]
    left = list(filter(lambda x: x < pivot, arr))
    right = list(filter(lambda x: x > pivot, arr))
    return quick_random(left, 0, len(left)-1) + [pivot] + quick_random(right, 0, len(right)-1)


if __name__ == "__main__":
    #1000, 10000, 100000, 1000000, 10000000
    n = 10000
    arr1 = random.sample(range(n), n)
    arr2 = arr1.copy()
    len_ = len(arr2) - 1
    
    #start = time.time()
    quick_pivot_first(arr1)
    #print(str(n)+" pivot first time: "+str(time.time() - start))
    
    start = time.time()
    #a = quick_random(arr2, 0, len_)
    print(str(n)+" pivot random time: "+str(time.time() - start))
    #print(a)

# PySpark

In [ ]:

from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate()
#sc = SparkContext("local", "QuickSort")

n = 100

#data = random.sample(range(n), n)
data = range(n)
rdd = sc.parallelize(data)

def quicksort(rdd, th):
    n = rdd.count()
    if n <= th:  
        return quick_pivot_first(rdd.collect())
        #return quick_random(rdd.collect(), 0, n-1)
        #return rdd.sortBy(lambda x: x).collect()
        #q = quick_random(rdd.collect(), 0, n-1)
    
    pivot = rdd.takeSample(False, 1)
    pivot = pivot[0] 
    left = rdd.filter(lambda x: x < pivot)
    right = rdd.filter(lambda x: x >= pivot)
    
    return quicksort(left, th) + quicksort(right, th)

start = time.time()
result = quicksort(rdd, max(rdd.count() * 0.3, 100))
end = time.time()
print(str(end - start))
with open("runtime.txt", "a") as f:
    f.write("Sorted data, first element as pivot\n max 100, n = "+str(n) +"\n")
    f.write(str(end - start) + "\n")

In [ ]:
#--------------Random data, Random element pivot----------------
# n = 10^3 : 256.50988364219666, max(rdd.count() * 0.2, 100)
# n = 10^3: 341.2896497249603, max(rdd.count() * 0.2, 10)

# n = 10^3 : 7.44262433052063, max(rdd.count() * 0.3, 1000)
# n = 10^4 : 107.05189633369446 , max(rdd.count() * 0.3, 1000)
# n = 10^5 : 230.27300763130188 , max(rdd.count() * 0.3, 1000)
# n = 10^6 : 217.59413886070251 , max(rdd.count() * 0.3, 1000)
# n = 10^7 : 309.6557824611664 , max(rdd.count() * 0.3, 1000)

#--------------Sorted, Random element pivot---------------------
# n = 10^3 : 12.57955551147461 , max(rdd.count() * 0.3, 1000)
# n = 10^4 : 135.81099581718445 , max(rdd.count() * 0.3, 1000)
# n = 10^5 : 103.72915506362915 , max(rdd.count() * 0.3, 1000)
# n = 10^6 : 169.46816682815552 , max(rdd.count() * 0.3, 1000)
# n = 10^7 : 183.5217707157135 , max(rdd.count() * 0.3, 1000)

#--------------Random data, First element pivot-----------------
# n = 10^3 : 7.416630268096924 , max(rdd.count() * 0.3, 1000)
# n = 10^4 : 158.62151956558228 , max(rdd.count() * 0.3, 1000)
# n = 10^5 : 157.78868794441223 , max(rdd.count() * 0.3, 1000)
# n = 10^6 : 142.17204546928406 , max(rdd.count() * 0.3, 1000)
# n = 10^7 : 272.95424938201904 , max(rdd.count() * 0.3, 1000)

In [ ]:
sc

In [ ]:
n = 1000

data = random.sample(range(n), n)
rdd = sc.parallelize(data)

In [ ]:
start = time.time()
#rdd.collect()
rdd.sortBy(lambda x:x).collect()
end = time.time()
print(str(end - start))

start = time.time()
#take = rdd.takeSample(False, n)
quick_random(rdd.collect(), 0, n-1)
end = time.time()
print(str(end - start))

In [ ]:
type(take)